In [1]:
import pandas as pd

from source.convert import eikon2wrds

In [2]:
eu_df = pd.read_csv("assets\Eikon\europe_data_matched.csv", low_memory=False)
eu_df

,Instrument,TotAssets Date,TotAssets.Year,PriceClose Date,PriceClose.Year,Company Name,Country of Incorporation,Country of Exchange,Country of Headquarters,Instrument Type,...,Total Liabilities,Loans & Receivables - Total,Net Cash Flow from Operating Activities,Income Taxes - Payable - Short-Term,Trade Account Payables - Total,Advertising Expense,Labor & Related Expenses - Total,Dividend Per Share - Actual,"EPS - Basic - excl Extraordinary Items, Common - Total",Price Close
0,AMAV.VI,2021-12-31,2021.0,2021-12-30,2021.0,AMAG Austria Metall AG,Austria,Austria,Austria,Ordinary Shares,...,9.642860e+08,199173000.0,45592000.0,20947000.0,107860000.0,NaN,175154000.0,NaN,1.85112,41.00
1,AMAV.VI,2020-12-31,2020.0,2020-12-30,2020.0,NaN,NaN,NaN,NaN,NaN,...,9.455910e+08,151050000.0,107287000.0,3728000.0,59111000.0,NaN,146618000.0,0.5,0.31287,29.90
2,AMAV.VI,2019-12-31,2019.0,2019-12-30,2019.0,NaN,NaN,NaN,NaN,NaN,...,8.823700e+08,157929000.0,139943000.0,10331000.0,73050000.0,NaN,157819000.0,2.4,1.09579,30.50
3,AMAV.VI,2018-12-31,2018.0,2018-12-28,2018.0,NaN,NaN,NaN,NaN,NaN,...,9.403690e+08,164438000.0,94288000.0,75000.0,89966000.0,NaN,147939000.0,1.2,1.26307,31.20
4,AMAV.VI,2017-12-31,2017.0,2017-12-29,2017.0,NaN,NaN,NaN,NaN,NaN,...,7.969950e+08,153077000.0,101841000.0,1036000.0,77564000.0,NaN,140232000.0,1.2,1.79106,51.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272301,BUCN.S,2014-12-31,2014.0,2014-12-30,2014.0,NaN,NaN,NaN,NaN,NaN,...,1.402900e+09,583300000.0,163400000.0,23500000.0,263600000.0,NaN,710400000.0,6.5,18.58665,248.90
272302,BUCN.S,2013-12-31,2013.0,2013-12-30,2013.0,NaN,NaN,NaN,NaN,NaN,...,1.362200e+09,509500000.0,223800000.0,49500000.0,261200000.0,NaN,662800000.0,5.0,19.63819,259.00
272303,BUCN.S,2012-12-31,2012.0,2012-12-28,2012.0,NaN,NaN,NaN,NaN,NaN,...,1.369100e+09,476700000.0,200100000.0,42200000.0,238400000.0,NaN,630100000.0,4.0,15.52675,180.00
272304,BUCN.S,2011-12-31,2011.0,2011-12-30,2011.0,NaN,NaN,NaN,NaN,NaN,...,1.433300e+09,496500000.0,171500000.0,33300000.0,272800000.0,NaN,585200000.0,3.0,12.52053,164.20


In [3]:
new_df = eikon2wrds(eu_df)
new_df

,Instrument,TotAssets Date,TotAssets.Year,PriceClose Date,PriceClose.Year,Company Name,Country of Incorporation,Country of Exchange,Country of Headquarters,Instrument Type,...,ibcom,itcb,mktcap,ni,ocf,oepsx,opeps,oprepsx,prcc_f,xad
0,AMAV.VI,2021-12-31,2021.0,2021-12-30,2021.0,AMAG Austria Metall AG,Austria,Austria,Austria,Ordinary Shares,...,65277000.0,0.0,1.445824e+09,64614000.0,45592000.0,1.85112,1.85109,1.85109,41.00,NaN
1,AMAV.VI,2020-12-31,2020.0,2020-12-30,2020.0,NaN,NaN,NaN,NaN,NaN,...,11033000.0,0.0,1.050274e+09,11059000.0,107287000.0,0.31287,0.32475,0.32475,29.90,NaN
2,AMAV.VI,2019-12-31,2019.0,2019-12-30,2019.0,NaN,NaN,NaN,NaN,NaN,...,38642000.0,0.0,1.075552e+09,38642000.0,139943000.0,1.09579,1.09579,1.09579,30.50,NaN
3,AMAV.VI,2018-12-31,2018.0,2018-12-28,2018.0,NaN,NaN,NaN,NaN,NaN,...,44541000.0,0.0,1.100237e+09,44541000.0,94288000.0,1.26307,1.26307,1.26307,31.20,NaN
4,AMAV.VI,2017-12-31,2017.0,2017-12-29,2017.0,NaN,NaN,NaN,NaN,NaN,...,63160000.0,0.0,1.812217e+09,63160000.0,101841000.0,1.79106,1.79106,1.79106,51.39,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272301,BUCN.S,2014-12-31,2014.0,2014-12-30,2014.0,NaN,NaN,NaN,NaN,NaN,...,187400000.0,0.0,2.551225e+09,189700000.0,163400000.0,18.58665,18.58665,18.50073,248.90,NaN
272302,BUCN.S,2013-12-31,2013.0,2013-12-30,2013.0,NaN,NaN,NaN,NaN,NaN,...,194500000.0,0.0,2.654750e+09,196200000.0,223800000.0,19.63819,19.63819,19.53222,259.00,NaN
272303,BUCN.S,2012-12-31,2012.0,2012-12-28,2012.0,NaN,NaN,NaN,NaN,NaN,...,151700000.0,0.0,1.845000e+09,156000000.0,200100000.0,15.52675,15.52675,15.45387,180.00,NaN
272304,BUCN.S,2011-12-31,2011.0,2011-12-30,2011.0,NaN,NaN,NaN,NaN,NaN,...,124700000.0,0.0,1.734921e+09,127400000.0,171500000.0,12.52053,12.52053,12.46448,164.20,NaN
